In [1]:
import sys
sys.path.insert(0, '../../../')

import pandas as pd
from preprocess import Audio_Processor
from data_utils import balanced_supersample, balanced_subsample
from sklearn import metrics
from classification_plots import plot_confusion_matrix, plot_learning_curve
import matplotlib.pyplot as plt
import os
import numpy as np
from data_utils import enumerate_strings
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Able to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1"

SR = 16000
blocksize = int(SR)
overlap = int(SR/2)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [4]:
path_to_db='../../../../ESC-50/'
ps = Audio_Processor(path_to_db + 'audio/', sr=SR)
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50
h_classes = ['Human & Animal', 'Interacting Materials']
mapping = {
    'dog': 0,
    'rooster': 0,
    'pig': 0,
    'cow': 0,
    'frog': 0,
    'cat': 0,
    'hen': 0,
    'insects': 0,
    'sheep': 0,
    'crow': 0,
    'rain': 1,
    'sea_waves': 1,
    'crackling_fire': 1,
    'crickets': 0,
    'chirping_birds': 0,
    'water_drops': 1,
    'wind': 1,
    'pouring_water': 1,
    'toilet_flush': 1,
    'thunderstorm': 1,
    'crying_baby': 0,
    'sneezing': 0,
    'clapping': 1,
    'breathing': 0,
    'coughing': 0,
    'footsteps': 1,
    'laughing': 0,
    'brushing_teeth': 1,
    'snoring': 0,
    'drinking_sipping': 1,
    'door_wood_knock': 1,
    'mouse_click': 1,
    'keyboard_typing': 1,
    'door_wood_creaks': 1,
    'can_opening': 1,
    'washing_machine': 1,
    'vacuum_cleaner': 1,
    'clock_alarm': 1,
    'clock_tick': 1,
    'glass_breaking':1,
    'helicopter': 1,
    'chainsaw': 1,
    'siren': 1,
    'car_horn': 1,
    'engine': 1,
    'train': 1,
    'church_bells': 1,
    'airplane': 1,
    'fireworks': 1,
    'hand_saw': 1,
}
dataset['h_target'] = None
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']
    dataset.loc[index, 'h_target'] = mapping[row['category']]

## Getting Preprocessed Data
We allow for previously preprocessed data to be retrieved for faster training turnaround. If the fold has been preprocessed, it is loaded but if not it is processed and saved.

In [ ]:
df = ps.preprocess_fold(dataset, 
                        kind='mfcc', 
                        blocksize=blocksize, 
                        overlap=overlap,
                        folds=10,
                        random_state=42
                       )

In [ ]:
df.describe()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise,h_target,target,fold
count,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,...,19427.000000,19427.000000,19427.000000,19427.000000,19427.000000,1.942700e+04,19427.000000,19427.000000,19427.000000,19427.000000
mean,65.298349,40.436487,32.685268,27.585829,24.377618,23.887475,23.391932,23.168676,22.842327,22.229750,...,-0.034276,-0.032712,18.811463,31.440679,7.034311,1.326578e+12,4.081588,0.665363,24.598291,5.428579
std,31.219519,18.579457,17.732584,15.832423,15.893643,16.263557,16.115932,16.647461,16.590921,16.327560,...,0.113828,0.110254,108.471522,79.455138,25.030786,8.268103e+13,3.836773,0.471876,13.921396,2.881739
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.418270,-0.415391,4.150735,6.184211,1.347528,0.000000e+00,0.335453,0.000000,0.000000,1.000000
25%,43.627568,27.085531,18.738593,14.524782,11.251145,10.627452,10.049943,9.725130,9.418548,9.137258,...,-0.111430,-0.107262,10.391233,19.426816,3.922225,1.580208e+01,1.476140,0.000000,13.000000,3.000000
50%,64.173310,39.940883,29.992873,23.933852,18.244478,16.883805,16.691123,15.339185,15.361837,14.277989,...,-0.004638,-0.003789,13.962201,24.287437,5.303062,2.026997e+01,3.134325,1.000000,25.000000,5.000000
75%,86.517212,53.066751,45.198926,40.461388,38.199421,38.669056,38.053225,38.523796,37.860079,37.266375,...,0.027792,0.024682,19.604827,33.260214,7.211585,2.931583e+01,5.520893,1.000000,36.000000,8.000000
max,228.345516,143.621780,104.833736,89.106989,90.419429,83.288488,103.937561,111.314025,118.082160,112.276576,...,0.400497,0.395673,13157.194138,6531.065989,2388.736017,5.154285e+15,98.250911,1.000000,49.000000,10.000000


In [ ]:
df.head()

,mfcc_2_std,mfcc_2_mean,mfcc_2_noise,mfcc_3_std,mfcc_3_mean,mfcc_3_noise,mfcc_4_std,mfcc_4_mean,mfcc_4_noise,mfcc_5_std,...,sflat_noise,sroll_std,sroll_mean,sroll_noise,rmse_std,rmse_mean,rmse_noise,h_target,target,fold
0,40.966990,26.258012,12.927575,13.748385,8.883775,5.561329,4.560276,7.306209,6.977689,7.903446,...,-0.401249,-0.356166,29.119574,25.008716,7.213768,31.710370,1.593856,0,0,6
1,35.142872,54.836846,41.036570,42.811786,43.850519,40.949468,45.337044,37.936349,48.159645,59.096694,...,0.211831,-0.137081,20.006969,17.513890,4.452054,23.070639,7.171350,0,14,9
2,57.409907,91.576618,45.413608,39.332333,53.766183,56.218496,64.165764,57.936285,60.855951,46.329327,...,0.062619,0.133189,16.410565,18.699163,4.967657,24.814506,7.261237,0,14,9
3,70.969104,72.930842,39.179481,38.887821,54.541822,47.095976,57.153255,49.467488,55.165133,47.755954,...,0.034366,-0.182873,17.590065,19.438896,4.188102,25.581114,4.904707,0,14,9
4,63.580718,69.414910,38.601297,36.367962,40.611029,45.260915,49.877285,49.425033,52.361329,46.373401,...,0.053480,-0.003533,17.617393,19.052473,3.936342,23.607172,4.998879,0,14,9


In [ ]:
from sklearn.model_selection import train_test_split
train, test, = train_test_split(
                        df[df.fold != 1], test_size=0.20, random_state=42)

In [ ]:
X = (df[df.fold != 1]).drop(['target', 'h_target', 'fold'], axis=1)
y = (df[df.fold != 1])['h_target']

In [ ]:
X, y = balanced_supersample(X, y)

# Training

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
import scipy

params = {
    'classify__n_neighbors': range(5,200,5),
    'classify__weights': ('uniform', 'distance'),
    'classify__algorithm': ('ball_tree','kd_tree','brute'),
    'classify__p': range(1,4)
}

clf = Pipeline([
    ('scaler',MinMaxScaler()),
    ('classify', KNN(n_jobs=5))
])
randsrc = RandomizedSearchCV(
                clf,
                scoring=['accuracy', 'f1', 'recall', 'precision', 'roc_auc'], 
                cv=10,
                refit=False,
                param_distributions = params,
                verbose=2,
                n_jobs=10,
                n_iter=100
)

randsrc.fit(X, y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:  8.4min
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed: 398.9min
[Parallel(n_jobs=10)]: Done 345 tasks      | elapsed: 1046.1min
[Parallel(n_jobs=10)]: Done 628 tasks      | elapsed: 1956.3min


In [ ]:
results = pd.DataFrame(randsrc.cv_results_)

In [ ]:
top = results.sort_values(['mean_test_roc_auc', 'mean_fit_time'], ascending=[False, True]).iloc[0]

In [ ]:
top.params

In [ ]:
clf.set_params(**top.params)

# Testing Optimal Classifier

### Training

In [ ]:
from sklearn.model_selection import train_test_split
train, test, = train_test_split(
                        df[df.fold != 1], test_size=0.20, random_state=42)

In [ ]:
train_X = train.drop(['target', 'h_target', 'fold'], axis=1)
train_y = train['h_target']
train_X, train_y = balanced_supersample(train_X, train_y)
test_X = test.drop(['target', 'h_target', 'fold'], axis=1)
test_y = test['h_target']

In [ ]:
clf.fit(train_X, train_y)

In [ ]:
pred = clf.predict(test_X)
print(metrics.accuracy_score(test_y, pred))
cm = metrics.confusion_matrix(test_y, pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, h_classes)
plt.show()

### Testing

In [ ]:
all_df = df[df.fold == 1]

In [ ]:
X = all_df.drop(['target', 'h_target', 'fold'], axis=1)
y = all_df['h_target']

In [ ]:
pred = clf.predict(X)
pred_prob = clf.predict_proba(X)
print(metrics.accuracy_score(y, pred))
cm = metrics.confusion_matrix(y, pred)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, h_classes)
plt.show()

### Confidence

In [ ]:
import seaborn as sns
misclassified = {
    'Predicted h_target': [],
    'Actual target': [],
    'Anim_prob': [],
    'Interact_prob': []
}
for row, p, prob, Y in zip(all_df.iterrows(), pred, pred_prob, y):
    Y = int(Y)
    p = int(p)
    if p!=Y:
        misclassified['Actual target'].append(classes[int(row[1]['target'])])
        misclassified['Predicted h_target'].append(h_classes[p])
        misclassified['Anim_prob'].append(prob[0])
        misclassified['Interact_prob'].append(prob[1])

misclassified = pd.DataFrame.from_dict(misclassified)
misclassified.head()

In [ ]:
gb_ms = misclassified.groupby('Predicted h_target')
means = gb_ms.mean()
errors = gb_ms.std()

In [ ]:
means.head()

In [ ]:
errors.head()

In [ ]:
means.min(axis=1).plot(yerr=errors, kind='bar')

In [ ]:
average_prob = misclassified.groupby('Actual target').mean()
average_prob

In [ ]:
average_prob.min(axis=1).plot(kind='bar')

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(50,10))
sns.countplot(x='Actual target', hue='Predicted h_target',data=misclassified)